<font face="Calibri" size="2"> <i>Open SAR Toolkit - Tutorial 1, version 1.3,  July 2020. Andreas Vollrath, ESA/ESRIN phi-lab</i>
</font>

![title](https://raw.githubusercontent.com/ESA-PhiLab/OpenSarToolkit/main/docs/source/_images/header_image.PNG)

--------

# OST Tutorial I 




## Pre-processing your first Sentinel-1 image with OST

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ESA-PhiLab/OST_Notebooks/blob/master/1%20-%20The%20first%20S1%20scene.ipynb)

--------

**Short description**

This notebook introduces you to the *Sentinel1Scene* class of the Open SAR Toolkit and demonstrates how it can be used to download, extract metadata and pre-process a single Sentinel-1 scene.

--------

**Requirements**

- a PC/Mac with at least 16GB of RAM
- about 4 GB of free disk space (or more if you want to process more scenes)
- a NASA Earthdata account with signed EULA for use of https://search.asf.alaska.edu (just register directly there)
--------

**NOTE:** all cells that have an * after its number can be executed without changing any code. 

### 0\* - Install OST and dependencies 

**NOTE:** Applies only if you haven't fully installed OST and its dependencies yet, e.g. on Google Colab, so uncomment the lines in this case.

In [ ]:
# !apt-get -y install wget
# !wget https://raw.githubusercontent.com/ESA-PhiLab/OST_Notebooks/master/install_ost.sh
# !bash install_ost.sh

### 1\* - Import the OST *Sentinel1Scene* class

In [ ]:
# these imports we need to handle the folders, independent of the OS
from pathlib import Path
from pprint import pprint

# this is the Sentinel1Scene class, that basically handles all the workflow from beginning to the end
from ost import Sentinel1Scene

# from ost.helpers.settings import set_log_level
# import logging
# set_log_level(logging.DEBUG)

### 2* - Create a folder for our outputs

By executing this cell, a new folder will be created and the path will be written to the *output_dir* variable

In [ ]:
# get home folder
home = Path.home()

# create a processing directory
output_dir = home / "OST_Tutorials" / "Tutorial_1"
output_dir.mkdir(parents=True, exist_ok=True)
print(str(output_dir))

### 3* - Choose scene ID and display some metadata

In order to initialize an instance of the *Sentinel1Scene* class, all we need is a valid scene id of a Sentinel-1 product. 

In [ ]:
# create a S1Scene class instance based on the scene identifier of the first ever Dual-Pol Sentinel-1 IW product

# ---------------------------------------------------
# Some scenes to choose from

# very first IW (VV/VH) S1 image available over Istanbul/Turkey
# NOTE:only available via ASF data mirror
scene_id = "S1A_IW_GRDH_1SDV_20141003T040550_20141003T040619_002660_002F64_EC04"

# other scenes with different scene types to process (uncomment)
# IW scene (dual-polarised HH/HV) over Norway/Spitzbergen
# scene_id = 'S1B_IW_GRDH_1SDH_20200325T150411_20200325T150436_020850_02789D_2B85'

# IW scene (single-polarised VV) over Ecuadorian Amazon
# scene_id = 'S1A_IW_GRDH_1SSV_20150205T232009_20150205T232034_004494_00583A_1C80'

# EW scene (dual-polarised VV/VH) over Azores (needs a different DEM, see cell of ARD parameters below)
# scene_id = 'S1B_EW_GRDM_1SDV_20200303T193150_20200303T193250_020532_026E82_5CE9'

# EW scene (dual-polarised HH/HV) over Greenland
# scene_id = 'S1B_EW_GRDM_1SDH_20200511T205319_20200511T205419_021539_028E4E_697E'

# Stripmap mode S5 scene (dual-polarised VV/VH) over Germany
# scene_id = 'S1B_S5_GRDH_1SDV_20170104T052519_20170104T052548_003694_006587_86AB'
# ---------------------------------------------------

# create an S1Scene instance
s1 = Sentinel1Scene(scene_id)

# print summarising infos about the scene
s1.info()

### 4* Download scene

The first step is to download the selected scene. In our case we chose the first regular Sentinel-1 IW product acquired in the dual-polarisation VV/VH acquired the 3rd of October 2014. 
OST provides download from 3 different mirrors, ESA's scihub, CNES PEPS and Alaska Satellite Facility (NASA Earthdata). Since ESA's official scihub became a rolling archive, and so is PEPS, best is to download from the fantastic **Alaska Satellite Facility** mirror (selection 2), which holds the full Sentinel-1 archive online. 

**Note I:** You can interrupt the download and restart it. The download will continue from where it stopped.

**Note II:** After the actual download, the file is checked for inconsistency. This assures that the download went fine and we can use it for processing. In addition, OST will magically remember that this file has been successfully downloaded (just run the cell again to see the behaviour).

In [ ]:
s1.download(output_dir)

### 5* - Define our ARD product

ARD stands for Analysis Ready Data and is interpreted differently by different persons. OST provides various pre-defined flavours which can be used instantly.

The following table shows the ARD types and corresponding processing steps applied for GRD data. 

|               ARD Types  | OST-GTC      | OST-RTC         | CEOS           | Earth-Engine |
| :-------------           | :----------: | :-----------:   | :----------:   | -----------: |
| **Scene processing steps**                                                                |        
|  Update Orbit File       |       x      |       x         |       x        |      x       |
|  Thermal Noise Removal   |       x      |       x         |       x        |      x       |
|  GRD Border Noise        |       x      |       x         |       x        |      -       |
|  Calibration             | $\gamma^0$   |   $\beta^0$     |  $\beta^0$     |  $\sigma^0$  |
|  Multi-look              |       x      |       x         |       -        |      -       |
|  Speckle-Filter          |       -      |       -         |       -        |      -       |
|  Terrain Flattening      |       -      | x ($\gamma^0_f$)|x ($\gamma^0_f$)|      -       |
|  Layover-Shadow Mask     |       -      |       x         |       x        |      -       |
|  dB conversion           |       -      |       -         |       -        |      x       |
|  Terrain Correction      |       x      |       x         |       x        |      x       |
|  Resolution (in m)       |       20     |       20        |       10       |      10      |

The default ARD type is *'OST-GTC'*, referring to a Geometrically Terrain Corrected product which is calibrated to the ellipsoid correceted $\gamma^0$ backscatter coefficient at 20m resolution. Other pre-defined ARD types are available, but it is also possible to customise single ARD parameters via a dictionary where all parameters are stored (as demonstrated in the cell below). Note how the resolution and the resampling of the image during terrain correction is changed at the bottom. In this way, actually all relevant parameters for processing are customisable. 

In [ ]:
# Default ARD parameter

print(
    "-----------------------------------------------------------------------------------------------------------"
)
print("Our ARD parameters dictionary contains 4 keys. For the moment, only single_ARD is relevant.")
print(
    "-----------------------------------------------------------------------------------------------------------"
)
pprint(s1.ard_parameters.keys())
print(
    "-----------------------------------------------------------------------------------------------------------"
)
print("")

print(
    "-----------------------------------------------------------------------------------------------------------"
)
print("Dictionary of our default OST ARD parameters for single scene processing:")
print(
    "-----------------------------------------------------------------------------------------------------------"
)
pprint(s1.ard_parameters["single_ARD"])
print(
    "-----------------------------------------------------------------------------------------------------------"
)
print("")

In [ ]:
# Template ARD parameters

# we change ARD type
# possible choices are:
# 'OST_GTC', 'OST-RTC', 'CEOS', 'Earth Engine'
s1.update_ard_parameters("Earth-Engine")
print(
    "-----------------------------------------------------------------------------------------------------------"
)
print("Dictionary of Earth Engine ARD parameters:")
print(
    "-----------------------------------------------------------------------------------------------------------"
)
pprint(s1.ard_parameters["single_ARD"])
print(
    "-----------------------------------------------------------------------------------------------------------"
)

In [ ]:
# Customised ARD parameters

# we cusomize the resolution and image resampling
s1.ard_parameters["single_ARD"]["resolution"] = 100  # set output resolution to 100m
s1.ard_parameters["single_ARD"]["remove_speckle"] = False  # apply a speckle filter
s1.ard_parameters["single_ARD"]["dem"][
    "image_resampling"
] = "BILINEAR_INTERPOLATION"  # BICUBIC_INTERPOLATION is default

# s1.ard_parameters['single_ARD']['product_type'] = 'RTC-gamma0'

# uncomment this for the Azores EW scene
# s1.ard_parameters['single_ARD']['dem']['dem_name'] = 'GETASSE30'
print(
    "-----------------------------------------------------------------------------------------------------------"
)
print("Dictionary of our customised ARD parameters for the final scene processing:")
print(
    "-----------------------------------------------------------------------------------------------------------"
)
pprint(s1.ard_parameters["single_ARD"])
print(
    "-----------------------------------------------------------------------------------------------------------"
)

### 6* - Create an ARD product

Our *Sentinel1Scene* class comes with the build-in method *create_ard* to produce a standardised ARD product based on the ARD dictionary above. 

To run the command we just have to provide: 
- the path to the downloaded file. We can use the *get_path* method in conjunction with the download directory provided
- a directory where the output files will be written to
- a filename prefix (the output format will be the standard SNAP Dimap format, consisting of the dim-file and the data directory)
- and a directory for storing temporary files (can not be the same as the output folder)

In [ ]:
s1.create_ard(infile=s1.get_path(output_dir), out_dir=output_dir, overwrite=True)

print(" The path to our newly created ARD product can be obtained the following way:")
s1.ard_dimap

### 6* - Create a RGB color composite

Sentinel-1 scenes usually consist of two polarisation bands. In order to create a 3 channel RGB composite a ratio between the Co- (VV or HH) and the Cross-polarised (VH or HV) band is added. The *create_rgb* method takes the *ard_dimap* file and converts it to a 3-channel GeoTiff.

In [ ]:
s1.create_rgb(outfile=output_dir / f"{s1.start_date}.tif")

print(" The path to our newly created RGB product can be obtained the following way:")
s1.ard_rgb

### 7* - Visualise the RGB composite

We can plot the newly created RGB image with the *visualise_rgb* method. A *shrink_factor* can be set, which reduces resolution in favour of memory requirements for plotting.

In [ ]:
# ---------------------------------------------------
# for plotting purposes we use this iPython magic
%matplotlib inline
%pylab inline
pylab.rcParams["figure.figsize"] = (23, 23)
# ---------------------------------------------------
s1.visualise_rgb(shrink_factor=2)

### 8* - Create thumbnail image

For some it might be interesting to create a small thumbnail image in Jpeg format. The *create_rgb_thumbnail* method allows for this. 

In [ ]:
# define a filename for our thumbnail image
path_to_thumbnail = output_dir / f"{s1.start_date}.TN.jpg"

# create the thumbnail image
s1.create_rgb_thumbnail(outfile=str(path_to_thumbnail))

In [ ]:
import imageio

img = imageio.imread(path_to_thumbnail)
!ls -sh {path_to_thumbnail}
plt.imshow(img)

### 9* - Play around

You can try out the different images and also check the difference in backscatter for RTC products (uncomment the line of product type in the customisable ARD parameters)